In [17]:
from langgraph.graph import StateGraph,START, END
from typing import TypedDict, Literal, Annotated
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage,BaseMessage
from langgraph.checkpoint.memory import MemorySaver
import uuid

In [18]:
from langgraph.graph.message import add_messages
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [19]:
llm = ChatOpenAI()

def chat_node(state: ChatState):

    messages = state['messages']
    response = llm.invoke(messages)
    return {'messages': [response]}


In [20]:
checkpointer = MemorySaver()
graph = StateGraph(ChatState)

graph.add_node("chat_node", chat_node)

graph.add_edge(START, "chat_node")
graph.add_edge("chat_node", END)

chatbot = graph.compile(checkpointer=checkpointer)

In [21]:
thread_id = str(uuid.uuid4())
while True:
    user_input = input("You: ")

    print("You:", user_input)
    
    if user_input.strip().lower() in ["quit", "exit", 'bye']:
        break
    config = {'configurable': {'thread_id': thread_id}}
    result = chatbot.invoke({
        'messages': [HumanMessage(content=user_input)]}, 
        config=config)
    
    print("Bot:", result['messages'][-1].content)

You: what is 10 + 10
Bot: 10 + 10 equals 20.
You: add 30 to it
Bot: Adding 30 to 20 equals 50.
You: what is capital of india
Bot: The capital of India is New Delhi.
You: what are major historical sites there
Bot: Some major historical sites in New Delhi, India include:

1. Red Fort
2. India Gate
3. Qutub Minar
4. Humayun's Tomb
5. Jama Masjid
6. Lotus Temple
7. Raj Ghat
8. Parliament House

These are just a few examples of the many historical sites in New Delhi.
You: what is capital of pakistan
Bot: The capital of Pakistan is Islamabad.
You: what historical things are there
Bot: Some major historical sites in Islamabad, Pakistan include:

1. Faisal Mosque
2. Pakistan Monument
3. Rawat Fort
4. Lok Virsa Museum
5. Shah Faisal Mosque
6. Daman-e-Koh
7. Pakistan National Museum
8. Rawalpindi Cricket Stadium

These are just a few examples of the historical sites in and around Islamabad.
You: where osama bin laden was found
Bot: Osama bin Laden was found and killed in a compound in Abbottabad